# Visualisation 

In [1]:
import numpy as np
import os
import cv2 as cv2
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [2]:
def show(img, factor=1,name="image"):
    """ 
    show an image until the escape key is pressed
    :param factor: scale factor (default 1, half size)
    """
    if factor != 1.0:
        img = cv2.resize(img, (0,0), fx=factor, fy=factor) 

    cv2.imshow(name,img)
    while(1):
        k = cv2.waitKey(0)
        if k==27:    # Esc key to stop
            break
    cv2.destroyAllWindows()

In [3]:
def read_image(input_path):
    img = cv2.imread(input_path,0)
    return img

## Dividing Paper

In [4]:
def preprocess_img(img,show_steps=1,show_size=0.2):
    img = cv2.GaussianBlur(img,(11,11),0)
    thresh = cv2.threshold(img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img[img >= thresh[0]] = 255
    img[img <= thresh[0]] = 0
    img = cv2.bitwise_not(img)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(2,2))
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    if show_steps == 1:        
        show(img,show_size,"Preprocessed Image")
    return img

In [5]:
def remove_top(img,show_steps=1,show_size=0.2):
    trsh = 0.1
    kernel = np.ones((5,5),np.uint8)
    erosion = cv2.dilate(img,kernel,iterations = 8)
    proj = np.sum(erosion,axis=1).astype(int)
    if show_steps == 1:        
        show(erosion,show_size,"Eroded")
    max_line = np.amax(proj)



    lines = []
    lines.append(300)
    for i in range(img.shape[0]):
        if(proj[i] >= trsh*max_line) :
            if(len(lines) > 0 ):
                if(lines[-1] + img.shape[0]/10 > i ):
                    continue
            if( (i>=500 and i <= 900) or (i>=2600 and i <= 2900)):
                lines.append(i)
    if show_steps == 1:        
        show(img[lines[1]:lines[2],:],show_size,"Cropped Image")
    
    return img[lines[1]:lines[2],:]


In [130]:
def get_sentences(img,show_steps=1,show_size=0.2):
    
    original = img.copy()
    blurred = cv2.GaussianBlur(img, (1, 1), 0)
    canny = cv2.Canny(blurred, 50, 255, 1)
    kernel = np.ones((1,5),np.uint8)
    dilate = cv2.dilate(canny, kernel, iterations=40)
    
    if show_steps == 1:        
        show(dilate,show_size,"Sentences")
    
    # Find contours
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    # Iterate thorugh contours and filter for ROI
    image_number = 0
    order_list = []
    images = []
    area = 0 
    for c in cnts:
        area += cv2.contourArea(c)
    area /= (len(cnts)+5)
    xarr = []
    rxarr = []
    yarr = []
    ryarr = []
    for c in cnts:
        if(cv2.contourArea(c) > area):
            x,y,w,h = cv2.boundingRect(c)
            if h > img.shape[0]/8:
                continue
            cv2.rectangle(img, (x, y), (x + w, y + h), (36,255,12), 2)
            ROI = original[y:y+h, x:x+w]
            if show_steps == 1:        
                show(ROI,show_size,"Sentence")
            images.append(ROI)
            order_list.append(y+x/1000)
            image_number += 1
    #print(max(xarr))
    segments = [x for _,x in sorted(zip(order_list,images))]
    return segments

In [131]:
def get_pixel(img, center, x, y): 
      
    new_value = 0
      
    try: 
        # If local neighbourhood pixel  
        # value is greater than or equal 
        # to center pixel values then  
        # set it to 1 
        if img[x][y] >= center: 
            new_value = 1
              
    except: 
        # Exception is required when  
        # neighbourhood value of a center 
        # pixel value is null i.e. values 
        # present at boundaries. 
        pass
      
    return new_value 
   
# Function for calculating LBP 
def lbp_calculated_pixel(img, x, y): 
   
    center = img[x][y] 
   
    val_ar = [] 
      
    # top_left 
    val_ar.append(get_pixel(img, center, x-1, y-1)) 
      
    # top 
    val_ar.append(get_pixel(img, center, x-1, y)) 
      
    # top_right 
    val_ar.append(get_pixel(img, center, x-1, y + 1)) 
      
    # right 
    val_ar.append(get_pixel(img, center, x, y + 1)) 
      
    # bottom_right 
    val_ar.append(get_pixel(img, center, x + 1, y + 1)) 
      
    # bottom 
    val_ar.append(get_pixel(img, center, x + 1, y)) 
      
    # bottom_left 
    val_ar.append(get_pixel(img, center, x + 1, y-1)) 
      
    # left 
    val_ar.append(get_pixel(img, center, x, y-1)) 
       
    # Now, we need to convert binary 
    # values to decimal 
    power_val = [1, 2, 4, 8, 16, 32, 64, 128] 
   
    val = 0
      
    for i in range(len(val_ar)): 
        val += val_ar[i] * power_val[i] 
          
    return val 
   

def get_LBP(sentence,show_steps=1,show_size=0.2):

    height, width = sentence.shape 


    # Create a numpy array as  
    # the same height and width  
    # of RGB image 
    img_lbp = np.zeros((height, width), np.uint8) 

    for i in range(0, height): 
        for j in range(0, width): 
            img_lbp[i, j] = lbp_calculated_pixel(sentence, i, j) 
    return img_lbp
#img = get_LBP(sentences[0])
#show(img)

In [163]:
def cropImage(image):
    xarr = []
    rxarr = []
    yarr = []
    ryarr = []
    for contour in contours:
        x,y,w,h = cv2.boundingRect(contour)
        if (x != 0):
            xarr.append(x)
            rxarr.append(x+w)
        if (y != 0):
            yarr.append(y)
            ryarr.append(y+h)
        if h > img.shape[0]/8:
            continue
    args = [min(xarr), max(rxarr), min(yarr), max(ryarr)]
    cropped = img[args[2]:args[3], args[0]:args[1]]
    return cropped

In [ ]:
# def divide(image):
#     factor = 3
#     rowInterval = image.shape[0]/3
#     colInterval = image.shape[1]/3
#     for i in range(0,factor):
#         for j in range(0, factor):
            

# Main

In [165]:
input_path = '/home/ahmad/Desktop/AlexRun/Pattern Recognition/Dataset/formsA-D/c02-026.png'
show_steps = 0
show_size = 0.2

images = []
#for file in os.listdir(input_path):
#    images.append(file)
#images.sort()
#for input_img in images:    
img = read_image(input_path)#+input_img)
img = preprocess_img(img,show_steps,show_size)
img = remove_top(img,show_steps,show_size)
original = img
kernel = np.ones((5,5))
dilation = cv2.dilate(img,kernel,iterations = 20)
sentences = get_sentences(img,show_steps,show_size)
cropped = cropImage(dilation)
#show(cropped, 0.2, "Cropped")
# for sentence in sentences:
#         contours, hierarchy = cv2.findContours(dilation.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#         for contour in contours:
#             x,y,w,h = cv2.boundingRect(contour)
#             cv2.rectangle(sentence, (x, y), (x + w, y + h), (255,255,255), 2)
#         show(contour, 0.2, "Cropped")

#contours, hierarchy = cv2.findContours(dilation.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
 
#print(contours[0])

#show(cropped, 0.3, "cropped")
#imgplot = plt.imshow(cropped,cmap='gray')
#plt.show()    #break


error: OpenCV(4.4.0) /tmp/pip-req-build-99ib2vsi/opencv/modules/imgproc/src/resize.cpp:3935: error: (-215:Assertion failed) !dsize.empty() in function 'resize'


In [159]:
#show(original, 0.5)